In [1]:
import numpy as np
import torch
import torch.nn  as nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
import os

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
def data_loader(data_dir, batch_size, random_seed = 42, shuffle=True,valid_size=0.1, test=False, NUM_WORKERS = os.cpu_count()):
    normalize = transforms.Normalize(
        mean= [0.4914, 0.4822, 0.4465],
        std = [0.2023, 0.1994, 0.2010])
    
    # define transforms
    transform = transforms.Compose([
        transforms.Resize(size= (224, 224)),
        transforms.ToTensor(),
        normalize])
    if test:
        dataset = datasets.CIFAR100(
            root=data_dir,
            train= False,
            download=True,
            transform=transform
        )

        data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, shuffle=shuffle, num_workers=NUM_WORKERS)

        return data_loader
    # load the datasets
    train_dataset = datasets.CIFAR100(
            root=data_dir,
            train= True,
            download=True,
            transform=transform
        )
    # load the datasets
    valid_dataset = datasets.CIFAR100(
            root=data_dir,
            train= True,
            download=True,
            transform=transform
        )
    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=NUM_WORKERS)
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=NUM_WORKERS)
    
    return (train_loader, valid_loader)
    
    

In [4]:
train_loader, valid_loader = data_loader(data_dir="./data", batch_size=64)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
test_loader = data_loader(data_dir="./data", batch_size=64, test=True)

Files already downloaded and verified


In [26]:
normalize = transforms.Normalize(
        mean= [0.4914, 0.4822, 0.4465],
        std = [0.2023, 0.1994, 0.2010])
transform = transforms.Compose([
        transforms.Resize(size= (224, 224)),
        transforms.ToTensor(),
        normalize])
train_dataset = datasets.CIFAR100(
            root="./data",
            train= True,
            download=True,
            transform=transform
        )

Files already downloaded and verified


In [29]:
num_classes = len(train_dataset.classes)

100

In [6]:
img, label = next(iter(test_loader))
img.shape

torch.Size([64, 3, 224, 224])

In [22]:
train_loader.classes

AttributeError: 'DataLoader' object has no attribute 'classes'

In [7]:
import torchvision.models as models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = models.vgg16(pretrained=True).to(device)
next(model_ft.parameters()).device

C:\Users\Nihar\anaconda3\envs\tf\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Nihar\anaconda3\envs\tf\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


device(type='cuda', index=0)

In [8]:
class VGG16(nn.Module):
    
    def __init__(self,num_classes= 10):
        super(VGG16, self).__init__()

        # (n - 2p + k / s) + 1 -> (224 - 2*1 +3 /1)+ 1 -> 226 *226*64
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        # (n - 2p + k / s) + 1 -> (226 - 2*1 +3 /1)+ 1 -> 228 *228*64
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2) # ((W-F+2*P )/S)+1 -> (228 - 2 + 2*0)/2+1 -> 114*114*64
        )

        # 
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2)
        )
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2)
        )
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2)
        )
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2)
        )
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7*7*512, 4096),
            nn.ReLU()
        )
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)
        x = self.layer9(x)
        x = self.layer10(x)
        x = self.layer11(x)
        x = self.layer12(x)
        x = self.layer13(x)
        x = x.reshape(x.size(0), -1) # (batch_height, width, num_channel) -> (64, 512, 7,7) -> (64,512*7*7)
        x = self.fc(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x
        
        
        

In [30]:
# Hyper parameters
num_classes = len(train_dataset.classes)
num_epochs = 20
batch_size = 16
learning_rate = 0.001

In [10]:
model_0 = VGG16(num_classes).to(device)

In [11]:
next(model_0.parameters()).device

device(type='cuda', index=0)

In [12]:
# Create a train step
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device ):
    """ Performs a training with a model trying to learn on data_loader"""
    
    # Training
    train_loss, train_acc = 0, 0

    # Put model into training mode
    model.train()
    
    # Add a loop to loop the training batches
    for batch, (X, y) in enumerate(data_loader):
        # Put data on target device
        X, y = X.to(device), y.to(device)
       
        # 1. Forward Pass
        y_pred = model(X)
    
        # 2. Calculate loss (per batch)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() # accumulate train loss
    
        # 3. Calcuate accuracy
        train_acc += accuracy_fn(y_true=y, y_pred= torch.argmax(torch.softmax(y_pred, dim=1), dim=1)) # go from logits -> prediction label 
        
        # 4. Optimizer zero grad
        optimizer.zero_grad()
    
        # 5. Loss backward
        loss.backward()
    
        # 6. Optimizer Step (Grdient desecent)
        optimizer.step()
    
    # Devide total train loss by length of train dataloader
    train_loss /= len(data_loader)
    
    # Calculate the test acc 
    train_acc /= len(data_loader)
    # Print out what's happening
    return train_loss, train_acc
    
    
    

In [13]:
def test_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn,
               device: torch.device = device ):
    
     """ Performs a testing with a model trying to learn on data_loader"""
     ## Testing
     test_loss, test_acc = 0, 0
     model.eval()
     with torch.inference_mode():
        for batch, (X_test, y_test) in enumerate(data_loader):
            # Put data on target device
            X_test, y_test = X_test.to(device), y_test.to(device)
            # 1. forward Pass
            test_pred = model(X_test)
    
            # 2. Calculate loss (accumulatively)
            test_loss += loss_fn(test_pred, y_test).item()
    
            # 3. Calcuate accuracy
            test_acc += accuracy_fn(y_true=y_test, y_pred= torch.argmax(torch.softmax(test_pred, dim=1), dim=1))  # go from logits -> prediction label 
    
        # Calculate test loss
        test_loss /= len(data_loader)
    
        # Calculate the test acc 
        test_acc /= len(data_loader)
    
        # Print out what's happening
        return test_loss, test_acc

In [14]:
torch.manual_seed(42)

def eval_model(model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device):
    """ Returns a dictionary containing the results of model predicting on data_loader."""

    loss, acc = 0 , 0
    model.eval()
    with torch.inference_mode():
        for X, y in tqdm(data_loader):
            # Put data on target device
            X, y = X.to(device), y.to(device)
            # Make prediction
            y_pred = model(X)
    
            # Accumulate the loss and acc values per batch
            loss += loss_fn(y_pred, y).item()
            acc += accuracy_fn(y_true=y,
                               y_pred = torch.argmax(torch.softmax(y_pred, dim=1), dim=1))
        # Scale loss and acc to find the average loss/acc per batch
        loss /= len(data_loader)
        acc /= len(data_loader)

    return {"model_name" : model.__class__.__name__, # only works whwn model was created with a class
            "model_loss" : loss.item(),
            "model_acc" : acc}


In [16]:
from tqdm.auto import tqdm

# 1. create a train function that takes in various model parameters + optimizer + loss
def train(model: torch.nn.Module,
          train_dataloader : torch.utils.data.DataLoader,
          test_dataloader : torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          accuracy_fn, epochs: int = 5):
    # 2. Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []}
    
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model_0,
                                          data_loader=train_dataloader,
                                          loss_fn=loss_fn,
                                          accuracy_fn=accuracy_fn,
                                          optimizer=optimizer,
                                          device= device)
        test_loss, test_acc = test_step(model=model_0,
                                          data_loader=test_dataloader,
                                          loss_fn=loss_fn,
                                          accuracy_fn=accuracy_fn,
                                          device= device)
        # Print out what' happening
        print(f"Epoch: {epoch} | Train loss: {train_loss:.4f} | Train Accuracy: {train_acc:.4f} | Test loss : {test_loss:.4f}| Test Accuracy:{test_acc:.4f}")
    
        # 5. update results dictonary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # 6. Return the filled results at the end of the epochs
    return results

In [33]:
# Creating loss function and Optimizer

loss_fn = nn.CrossEntropyLoss() # measure how wrong our model is

optimizer = torch.optim.SGD(params= model_0.parameters(), lr = 0.1) 

def accuracy_fn(y_true, y_pred):
    """ Calculate accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 99.9999
    """
    correct = torch.eq(y_true, y_pred).sum().item()

    acc = (correct/len(y_pred)) * 100
    return acc

In [34]:
# Set random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# set number of epochs
NUM_EPOCHS = 5

# Recreate an instnce of TinyVGG
model_0 = VGG16(num_classes= num_classes).to(device)

# Creating loss function and Optimizer

loss_fn = nn.CrossEntropyLoss() # measure how wrong our model is

optimizer = torch.optim.Adam(params= model_0.parameters(), lr = 0.001) 

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_0_results = train(model=model_0,
                       train_dataloader=train_loader,
                       test_dataloader=test_loader,
                       optimizer =optimizer,
                       loss_fn = loss_fn,
                       accuracy_fn=accuracy_fn, 
                       epochs= NUM_EPOCHS)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/5 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 196.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 7.08 GiB is allocated by PyTorch, and 179.09 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)